In [ ]:
# Importing the necessary libraries
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import mlflow.sklearn

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np


In [ ]:
# Loading dataset
df = pd.read_csv('../data/ssma.csv')

In [ ]:
def preprocess_data(df):
    # Dropping Student ID as it is not required
    df = df.drop(columns=['Student ID'], errors='ignore')
    
    # Encode binary categorical variables
    df['Gender'] = df['Gender'].map({'Male': 0, 'Female': 1})
    df['Affects_Academic_Performance'] = df['Affects_Academic_Performance'].map({'Yes': 1, 'No': 0})
    
    # List of categorical variables for one-hot encoding
    categorical_vars = ['Academic_Level', 'Country', 'Most_Used_Platform', 'Relationship_Status']
    
    # Numerical variables
    numerical_vars = [
        'Age', 'Avg_Daily_Usage_Hours', 'Sleep_Hours_Per_Night', 'Mental_Health_Score'
    ]
    
    # Defining transformers
    numeric_transformer = StandardScaler()
    categorical_transformer = OneHotEncoder(handle_unknown='ignore')
    
    # Preprocessing pipeline
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numerical_vars),
            ('cat', categorical_transformer, categorical_vars)
        ])
    
    X = df.drop(columns=['Conflicts_Over_Social_Media', 'Addicted_Score'])
    y_conflict = df['Conflicts_Over_Social_Media']
    y_addiction = df['Addicted_Score']
    
    X_processed = preprocessor.fit_transform(X)
    
    return X_processed, y_conflict, y_addiction


In [4]:
X_processed, y_conflict, y_addiction = preprocess_data(df)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_processed, y_addiction, test_size = 0.2, random_state = 0)

In [6]:
def report_dict(y_true, y_pred):
    return {
        "MAE": mean_absolute_error(y_true, y_pred),
        "MSE": mean_squared_error(y_true, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_true, y_pred)),
        "R2": r2_score(y_true, y_pred)
    }

### Experiment 1: Training basic LinearRegression 

In [7]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred_lin_reg = lin_reg.predict(X_test)
#print(report_dict(y_test, y_pred_lin_reg))


### Experiment 2: Training RandomForest Regression

In [8]:

from sklearn.ensemble import RandomForestRegressor

rf_reg = RandomForestRegressor(n_estimators = 10, random_state = 0) 
rf_reg.fit(X_train, y_train)
y_pred_rf_reg = rf_reg.predict(X_test)
#print(report_dict(y_test, y_pred_rf_reg))

### Experiment 3: Training XGBoost Regression

In [9]:
from xgboost import XGBRegressor

xgb_reg = XGBRegressor(max_depth = 2, learning_rate = 0.1, n_estimators = 100)
xgb_reg.fit(X_train, y_train)
y_pred_xgb_reg = xgb_reg.predict(X_test)
#print(report_dict(y_test, y_pred_xgb_reg))

### Experiment 4: Training SVR Regression

In [10]:
from sklearn.svm import SVR

# Create pipeline (scaling is important for SVR)
pipeline = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),
    ("svr", SVR(kernel='rbf', C=1.0, epsilon=0.2))
])
pipeline.fit(X_train, y_train)
y_pred_svr_reg = pipeline.predict(X_test)
#print(report_dict(y_test, y_pred_svr_reg))

### Tracking  Experiments Using MLFlow

In [11]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

# setting up the different models for the experiments
models = [
    (
        "Linear Regression", 
        LinearRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        RandomForestRegressor(n_estimators = 10, random_state = 0), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBoost",
        XGBRegressor(max_depth = 2, learning_rate = 0.1, n_estimators = 100), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "SVR",
        SVR(kernel = 'rbf'),
        (X_train, y_train),
        (X_test, y_test)
    )
]

In [12]:
reports = []

for model_name, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = report_dict(y_test, y_pred)
    reports.append(report)

In [ ]:
# Initializing MLflow

mlflow.end_run()

mlflow.set_experiment("addiction_level_regressor_v2")
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000/")

def adjusted_r2_score(y_true, y_pred, n_features):
    r2 = r2_score(y_true, y_pred)
    n = len(y_true)
    return 1 - (1 - r2) * (n - 1) / (n - n_features - 1)

def regression_accuracy(y_true, y_pred, tolerance=0.1):
    return np.mean(np.abs(y_true - y_pred) <= tolerance)

for i, element in enumerate(models):
    model_name = element[0]
    model = element[1]
    X_test = element[3][0]
    y_test = element[3][1]
    y_pred = model.predict(X_test)
    report = report_dict(y_test, y_pred)
    n_features = X_test.shape[1] if hasattr(X_test, 'shape') else X_test.shape[0]
    adj_r2 = adjusted_r2_score(y_test, y_pred, n_features)
    acc_01 = regression_accuracy(y_test, y_pred, 0.1)
    acc_05 = regression_accuracy(y_test, y_pred, 0.5)
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_param("model", model_name)
        mlflow.log_metric('MAE', report["MAE"])
        mlflow.log_metric('MSE', report["MSE"])
        mlflow.log_metric('RMSE', report["RMSE"])
        mlflow.log_metric('R²', report["R2"])
        mlflow.log_metric('Adjusted_R²', adj_r2)
        mlflow.log_metric('Accuracy_within_0.1', acc_01)
        mlflow.log_metric('Accuracy_within_0.5', acc_05)
        
         # Use model-specific artifact names
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, f"{model_name.lower().replace(' ', '_')}_model")
        else:
            mlflow.sklearn.log_model(model, f"{model_name.lower().replace(' ', '_')}_model")

        # # Register model in registry for all models
        # try:
        #     run_id = mlflow.active_run().info.run_id
        #     model_uri = f"runs:/{run_id}/model"
        #     mlflow.register_model(model_uri, "addiction_level_ regressor")
        #     print(f"✅ Successfully registered {model_name} model")
        # except Exception as e:
        #  print(f"⚠️  Warning: Could not register {model_name} model: {e}")

2025/07/20 19:33:21 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/20 19:33:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Linear Regression at: http://127.0.0.1:5000/#/experiments/563191459344469872/runs/b5e9f3320b4740458d32e0a571e22932
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/563191459344469872


2025/07/20 19:33:30 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/20 19:33:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/563191459344469872/runs/7b44dbbc09734b37be1d23ae8cf0f2fd
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/563191459344469872


2025/07/20 19:33:35 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Users\kola_\Documents\projects\SDS-CP029-social-sphere\.venv\Lib\site-packages\xgboost\sklearn.py:1028: UserWarning: [19:33:36] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
2025/07/20 19:33:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBoost at: http://127.0.0.1:5000/#/experiments/563191459344469872/runs/75b7b4eb7116473f9806de9110655721
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/563191459344469872


2025/07/20 19:33:41 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/20 19:33:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run SVR at: http://127.0.0.1:5000/#/experiments/563191459344469872/runs/69379c4832bc49ca8c2f3649277cd92f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/563191459344469872


### Compairing the Models

In [15]:
# Compare all models
print("📊 Model Performance Comparison:")
print("=" * 50)

comparison_data = []
for i, element in enumerate(models):
    model_name = element[0]
    model = element[1]
    X_test = element[3][0]
    y_test = element[3][1]
    y_pred = model.predict(X_test)
    report = report_dict(y_test, y_pred)
    
    comparison_data.append({
        'Model': model_name,
        'R² Score': report['R2'],
        'RMSE': report['RMSE'],
        'MAE': report['MAE']
    })

# Create comparison dataframe
comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('R² Score', ascending=False)

print(comparison_df.to_string(index=False))

# Find best model
best_model = comparison_df.iloc[0]
print(f"\n🏆 Best Performing Model: {best_model['Model']}")
print(f"   R² Score: {best_model['R² Score']:.4f}")
print(f"   RMSE: {best_model['RMSE']:.4f}")

📊 Model Performance Comparison:
            Model  R² Score     RMSE      MAE
    Random Forest  0.984939 0.194608 0.068085
              SVR  0.975717 0.247106 0.148652
          XGBoost  0.970424 0.272711 0.188535
Linear Regression  0.949345 0.356896 0.247683

🏆 Best Performing Model: Random Forest
   R² Score: 0.9849
   RMSE: 0.1946
